In [380]:
import sys
import math
import string
import operator
import random
import collections
import datetime
import itertools
import functools

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import IPython
import sympy as sp
import sympy.core.random as spr
import sympy.plotting as splt
import sympy.physics.vector as spv
import sympy.physics.mechanics as spm
import scipy.constants as spc

import IPython.display as ipd
spv.init_vprinting()
%matplotlib inline

HALF = sp.S.Half
PI = sp.pi
E = sp.exp
POSITIVEINFINITY = sp.S.Infinity


def reference_frame(
    frame: str,
    x=r'\imath', y=r'\jmath', z=r'\mathbf k'
) -> spv.ReferenceFrame:
    return spv.ReferenceFrame(
        frame, latexs=(
            fr'\; {{}}^\mathcal {frame} \hat {x}',
            fr'\;{{}}^\mathcal {frame} \hat {y}',
            fr'\: {{}}^\mathcal {frame} \hat {{z}}'
        )
    )


def RAOreference_frame(
    frame: str,
    x=r'E_1', y='E_2', z=r'E_3'
) -> spv.ReferenceFrame:
    return spv.ReferenceFrame(
        frame, latexs=(
            fr'\; {{}}^{frame} \hat {x}',
            fr'\;{{}}^{frame} \hat {y}',
            fr'\: {{}}^{frame} \hat {z}'
        )
    )


def vector(F: spv.ReferenceFrame, rx, ry, rz=0) -> spv.Vector:
    return rx*F.x + ry*F.y + rz*F.z


def vector_cos(
    F: spv.ReferenceFrame, magnitude,
    anglex, angley, anglez=sp.pi/2
) -> spv.Vector:
    return (magnitude *
            (sp.cos(anglex)*F.x
             + sp.cos(angley)*F.y
             + sp.cos(anglez)*F.z
             )
            )


def vector_line(
    start: spv.Vector, finish: spv.Vector
) -> spv.Vector:
    return finish - start


def vector_line_eqn(
    F: spv.ReferenceFrame, start: spv.Vector, finish: spv.Vector, kappa
) -> spv.Vector:
    return start + vector_line(start, finish).normalize()*kappa


def angle_between_vectors(a: spv.Vector, b: spv.Vector):
    return sp.acos(a.dot(b)/a.magnitude()/b.magnitude())


def print_aligned_latex_equations(*args):
    result = r'\\'.join([
        r'\begin{equation}',
        r'\begin{split}',
        *args,
        r'\nonumber',
        r'\end{split}',
        r'\end{equation}'
    ])

    ipd.display(ipd.Math( rf'{result}'))

__Mr Hart Tweet__

![](./images/MrHartTweet.png)

In [381]:
a, b, hx, hy, g = sp.symbols("a, b, h_x, h_y, g", real=True, positive=True)
values = {g:9.8, a:sp.Float(52.5), b:sp.Float(50), hy:10}
values

## One frame solution 1

__Kinematics__

In [382]:
t = sp.symbols('t', real=True)
vix, viy = sp.symbols("v_ix, v_iy", real=True)
rf = sp.Matrix([b, -a, 0])
r_max = sp.Matrix([hx, hy, 0])

v = sp.Matrix([vix, viy - g*t, 0])
r = sp.Matrix([vix*t, viy*t - HALF*g*t**2, 0])

assert (r.diff(t) - v == sp.zeros(3,1))

r, r_max

⎛⎡     t⋅vᵢₓ     ⎤       ⎞
⎜⎢               ⎥       ⎟
⎜⎢     2         ⎥  ⎡hₓ ⎤⎟
⎜⎢  g⋅t          ⎥  ⎢   ⎥⎟
⎜⎢- ──── + t⋅v_iy⎥, ⎢h_y⎥⎟
⎜⎢   2           ⎥  ⎢   ⎥⎟
⎜⎢               ⎥  ⎣ 0 ⎦⎟
⎝⎣       0       ⎦       ⎠

In [383]:
eqn1 = sp.Eq(v, sp.Matrix([vix, 0, 0]))
eqn2 = sp.Eq(r, r_max )
eqn1, eqn2

⎛                       ⎡     t⋅vᵢₓ     ⎤        ⎞
⎜                       ⎢               ⎥        ⎟
⎜⎡    vᵢₓ    ⎤   ⎡vᵢₓ⎤  ⎢     2         ⎥   ⎡hₓ ⎤⎟
⎜⎢           ⎥   ⎢   ⎥  ⎢  g⋅t          ⎥   ⎢   ⎥⎟
⎜⎢-g⋅t + v_iy⎥ = ⎢ 0 ⎥, ⎢- ──── + t⋅v_iy⎥ = ⎢h_y⎥⎟
⎜⎢           ⎥   ⎢   ⎥  ⎢   2           ⎥   ⎢   ⎥⎟
⎜⎣     0     ⎦   ⎣ 0 ⎦  ⎢               ⎥   ⎣ 0 ⎦⎟
⎝                       ⎣       0       ⎦        ⎠

In [384]:
result_max_height = sp.solve(
    [eqn1.subs(values), eqn2.subs(values)], 
    [t,  vix, viy]
)
result_max_height

In [385]:
tmax = sp.Symbol('t_max', real=True, negative=False)

for time, vx, vy in result_max_height:
    if time > 0:
        values.update({tmax:time, vix:vx, viy:vy})

In [386]:
tf = sp.Symbol('t_f', real=True, negative=False)
eqn3 = sp.Eq(rf, r.subs(t, tf).subs(values))
eqn3

       ⎡     0.7⋅hₓ⋅t_f      ⎤
⎡b ⎤   ⎢                     ⎥
⎢  ⎥   ⎢         2           ⎥
⎢-a⎥ = ⎢- 4.9⋅t_f  + 14.0⋅t_f⎥
⎢  ⎥   ⎢                     ⎥
⎣0 ⎦   ⎣          0          ⎦

In [387]:
result_tf = sp.solve(eqn3.subs(values), [tf, hx], dict=True)[0]
result_tf

In [388]:
values.update(result_tf)

In [389]:
values

In [390]:
sp.sqrt(values[vix].subs(values)**2 + values[viy]**2)

In [391]:
rf.normalized()

⎡     b      ⎤
⎢────────────⎥
⎢   _________⎥
⎢  ╱  2    2 ⎥
⎢╲╱  a  + b  ⎥
⎢            ⎥
⎢    -a      ⎥
⎢────────────⎥
⎢   _________⎥
⎢  ╱  2    2 ⎥
⎢╲╱  a  + b  ⎥
⎢            ⎥
⎣     0      ⎦

In [392]:
rf.normalized().subs(values)

⎡0.689655172413793 ⎤
⎢                  ⎥
⎢-0.724137931034483⎥
⎢                  ⎥
⎣        0         ⎦

In [393]:
v

⎡    vᵢₓ    ⎤
⎢           ⎥
⎢-g⋅t + v_iy⎥
⎢           ⎥
⎣     0     ⎦

In [394]:
values

In [395]:
t_norm = sp.symbols('t_norm', real=True, negative=False)
v.normalized().subs(values).subs(values).subs(t, t_norm)

⎡            0.714285714285714            ⎤
⎢─────────────────────────────────────────⎥
⎢   ______________________________________⎥
⎢  ╱                2                     ⎥
⎢╲╱  (0.7⋅tₙₒᵣₘ - 1)  + 0.510204081632653 ⎥
⎢                                         ⎥
⎢  0.0714285714285714⋅(14.0 - 9.8⋅tₙₒᵣₘ)  ⎥
⎢─────────────────────────────────────────⎥
⎢   ______________________________________⎥
⎢  ╱                2                     ⎥
⎢╲╱  (0.7⋅tₙₒᵣₘ - 1)  + 0.510204081632653 ⎥
⎢                                         ⎥
⎣                    0                    ⎦

In [396]:
eqn4 = sp.Eq(
    rf.normalized().subs(values), 
    v.normalized().subs(values).subs(values).subs(t, t_norm)
)

eqn4

                       ⎡            0.714285714285714            ⎤
                       ⎢─────────────────────────────────────────⎥
                       ⎢   ______________________________________⎥
                       ⎢  ╱                2                     ⎥
⎡0.689655172413793 ⎤   ⎢╲╱  (0.7⋅tₙₒᵣₘ - 1)  + 0.510204081632653 ⎥
⎢                  ⎥   ⎢                                         ⎥
⎢-0.724137931034483⎥ = ⎢  0.0714285714285714⋅(14.0 - 9.8⋅tₙₒᵣₘ)  ⎥
⎢                  ⎥   ⎢─────────────────────────────────────────⎥
⎣        0         ⎦   ⎢   ______________________________________⎥
                       ⎢  ╱                2                     ⎥
                       ⎢╲╱  (0.7⋅tₙₒᵣₘ - 1)  + 0.510204081632653 ⎥
                       ⎢                                         ⎥
                       ⎣                    0                    ⎦

In [397]:
sp.solve(eqn4, t_norm)